

 # **Ejercicios prácticos de acceso vía Entrez/E-utilities**

In [1]:
from typing import Optional
from io import StringIO
import json
import requests
import jsonpath_ng.ext as jp
import pandas as pd
from Bio import SeqIO
from lxml import etree
import time
import matplotlib.pyplot as plt

EInfo = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/einfo.fcgi"
ESearch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
ESummary = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
EFetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

In [2]:
def get_einfo(db:Optional[str]= None) -> dict:
    params = { "retmode": "json" }
    if db:
        params.update({"db": db})
    response = requests.get(EInfo, params=params, timeout=10)
    response.raise_for_status()
    return response.json()

In [3]:
def eSearch(term: str, db: str, retstart=0, retmax=20) -> dict:
    params = {
        "retmode": "json",
        "db":db,
        "term": term,
        "retstart": retstart,
        "retmax": retmax
    }
    response = requests.get(ESearch, params=params, timeout=10)
    response.raise_for_status()
    return response.json()

In [4]:
def eSummary(id:str, db:str) -> dict:
    params = {
        "id": id,
        "retmode": "json",
        "db": db
    }
    response = requests.get(ESummary, params=params, timeout=20)
    response.raise_for_status()
    return response.json()

In [5]:
def efetch(id: str, db: str, format: str):
    if format == "xml":
        retmode = 'xml'
        rettype = None
    if format == "genbank":
        retmode = "text"
        rettype = "gb"
    if format == "fasta":
        retmode = "text"
        rettype = "fasta"
    if format == "ft":
        retmode = "text"
        rettype = "ft"
    params = {
        "db": db,
        "id": id,
        "retmode": retmode,
        "rettype": rettype
    }
    response = requests.get(EFetch, params=params, timeout=10)
    response.raise_for_status()
    return response.content.decode()

# 1. **EInfo: buscar todas las bases de datos disponibles**
 Genera una tabla con:
 - Nombre de la base de datos
 - Descripción
 - Número de registros
 - Fecha de última Actualización.

In [ ]:
all_dbs = [ x.value for x in jp.parse("$..dblist[*]").find(get_einfo()) ]

In [ ]:
all_dbs_data = []
for current_db in all_dbs:
    data = get_einfo(current_db)
    dbname = jp.parse("$..dbname").find(data)[0].value
    dbdescription = jp.parse("$..description").find(data)[0].value
    dbcount = jp.parse("$..count").find(data)[0].value
    dblastupdate = jp.parse("$..lastupdate").find(data)[0].value
    all_dbs_data.append([dbname, dbdescription, dbcount, dblastupdate])

all_dbs_table = pd.DataFrame(
    all_dbs_data,
    columns=["name", "dbdescription", "count", "lastupadte"]
)

In [ ]:
display(all_dbs_table)



 ## 2. **Buscar el ID de una secuencia de GenBank por nombre**
 Usa `esearch` para encontrar el **ID** de la secuencia del gen **BRCA1
 humano** en GenBank.

In [ ]:
term = 'BRCA1[Gene Name] AND "Homo sapiens"[Organism] AND BRCA1[Title] AND (biomol_genomic[PROP] AND refseq[filter])'
data = eSearch(term, "nuccore")

n_results = jp.parse("$..count").find(data)[0].value
print(f"Se encontraron: {n_results} registros.")
print("El id es:", jp.parse("$..idlist[0]").find(data)[0].value)



 ## 3. **Obtener el archivo GenBank completo para un Accession ID**
 Usa `efetch` para descargar el archivo GenBank (`retmode=gb`) del gen
 **COX1** de *Homo sapiens*.

In [ ]:
# Hay dos genes COX1 humanos, uno en cromosoma 9 y otro en mitocondria
# La cromosomica se llama oficialmente PTGS1 y la mitocondrial MT-CO1

term = 'COX1[Gene Name] AND "Homo sapiens"[Organism] AND PTGS1[Title] AND (biomol_genomic[PROP] AND refseq[filter])'
data = eSearch(term, "nuccore")

n_results = jp.parse("$..count").find(data)[0].value
print(f"Se encontraron: {n_results} registros.")
current_id = jp.parse("$..idlist[0]").find(data)[0].value
print("El id es:", current_id)

In [ ]:
gb = efetch(current_id, "nuccore", "genbank")

with open("cox1.genbank", "w", encoding="utf-8") as fout:
    fout.write(gb)



 ## 4. **Descargar la secuencia en formato FASTA**
 Busca y descarga la secuencia en formato **FASTA** del gen **16S rRNA** de
 *Escherichia coli*.

In [ ]:
term = '"16S rRNA"[Gene Name] AND "Escherichia coli"[Organism] AND 16S[Title] NOT partial[title] AND biomol_genomic[PROP]'

search = eSearch(term, "nuccore")

n_results = jp.parse("$..count").find(search)[0].value
print(f"Se encontraron: {n_results} registros.")
current_ids = [x.value for x in jp.parse("$..idlist[*]").find(search) ]
print("Los Ids son:", current_ids)

In [ ]:
result = efetch(",".join(current_ids), db="nuccore", format="fasta")
with open("16s.fasta", "w", encoding="utf8") as fout:
    fout.write(result)




 ## 5. **Listar todos los Accessions de un organismo**
 Usa `esearch` + `efetch` para listar los **primeros 20 Accessions** de
 secuencias de *Arabidopsis thaliana* en RefSeq.

In [ ]:
term = 'Arabidopsis thaliana[Organism] and srcdb_refseq'

search = eSearch(term, "nuccore")

for c_id in jp.parse("$..idlist[*]").find(search):
    print(f"Id: {c_id.value}")



 ## 6. **Obtener los metadatos de una secuencia**
 Con `esummary`, extrae información como el título, longitud y fecha de
 publicación para una secuencia dada.

In [ ]:
def get_metadata(id:str):
    data = eSummary(id, "nuccore")
    title = jp.parse("$..title").find(data)[0].value
    sequence_length = jp.parse("$..slen").find(data)[0].value
    create_date = jp.parse("$..createdate").find(data)[0].value
    return {
        "title": title,
        "sequence_length": sequence_length,
        "create_date": create_date
    }

In [ ]:
get_metadata("1162502436")



 ## 7. **Buscar genes por palabra clave en la definición**
 Busca secuencias que contengan la palabra "photosystem II" en su definición y
 recupera los primeros 10 Accessions.

In [ ]:
# La definición en un archivo Genbank es lo que muestra como título.
term = '"photosystem II"[Title]'
search = eSearch(term, "nuccore")


n_results = jp.parse("$..count").find(search)[0].value
print(f"Se encontraron: {n_results} registros.")

current_ids = [x.value for x in jp.parse("$..idlist[*]").find(search)][:10]
print("Los 10 primeros Ids son:", current_ids)


In [ ]:

summaries = eSummary(",".join(current_ids), "nuccore")


summaries

In [ ]:
id_acc = []
for c_id in current_ids:
    query = f'$.result.["{c_id}"].accessionversion'
    accession = jp.parse(query).find(summaries)[0].value
    id_acc.append([c_id, accession])

id_acc_table = pd.DataFrame(
    id_acc,
    columns=["ID", "Accession"]
)
display(id_acc_table)



 ## 8. **Filtrar por tipo de molécula**
 Busca secuencias de ARN mensajero (**mRNA**) en *Mus musculus* relacionadas
 con "interleukin".

In [ ]:
term = '"Mus musculus"[Organism] AND interleukin[Title] AND biomol_mrna[PROP]'

search = eSearch(term, "nuccore")

count = jp.parse("$..count").find(search)[0].value

print(f"Count: {count}")

ids = [x.value for x in  jp.parse("$..idlist[*]").find(search)]

summaries = eSummary(",".join(ids), "nuccore")

for title in jp.parse("$..title").find(summaries):
    print("Title: ", title.value)

In [ ]:
fetch = efetch(",".join(ids), "nuccore", format="fasta")
print(fetch)



 ## 9. **Obtener la traducción de una proteína a partir del CDS**
  Extrae el campo `/translation` de una entrada GenBank que contenga una
  secuencia codificante (CDS).

In [ ]:
def extract_translations(prot_id:str):
    fetch = efetch(prot_id, "nuccore", "genbank")
    translations = []
    for record in SeqIO.parse(StringIO(fetch), "genbank"):
        cdss = [
            f
            for f in record.features
            if f.type == "CDS"
        ]
        c_translations = [
            tr
            for cds in cdss
            if "translation" in cds.qualifiers
            for tr in cds.qualifiers["translation"]
        ]
        for tr in c_translations:
            translations.append(tr)
    return translations

extract_translations(12545)



 ## 10 **Buscar todos los genomas completos de un organismo**
  Encuentra todas las entradas de genomas **completos** de *Mycobacterium
  tuberculosis*.

In [ ]:
term = '"Mycobacterium tuberculosis"[Organism] and "complete genome"[title]'
received_ids = []
total_received = 0
count = 1
tries_left = 100
while total_received < count and tries_left>0:
    tries_left -= 1
    search = eSearch(term, "nuccore", retstart=total_received, retmax=100)
    count = int(jp.parse("$..count").find(search)[0].value)
    current_ids = [x.value for x in jp.parse("$..idlist[*]").find(search)]
    received_ids.extend(current_ids)
    total_received += len(current_ids)


print(f"Recupere: {len(received_ids)} ids.")



 ## 11. **Extraer estadísticas de longitud**
  Descarga 100 secuencias de genes de *Saccharomyces cerevisiae* y calcula la
  distribución de longitudes.

In [ ]:
term = '"Saccharomyces cerevisiae"[Organism]'

search = eSearch(term, "gene", retmax=100)

identifiers = [x.value for x in jp.parse("$..idlist[*]").find(search)]

print(identifiers)

In [ ]:
all_data ={}
for identifier in identifiers:
    time.sleep(1)
    data = efetch(identifier, "gene", format="xml")
    all_data[identifier] = data

In [ ]:

rows = []
for cid, c_data in all_data.items():
    root = etree.parse(StringIO(c_data))
    seq_accession = root.xpath(".//Entrezgene_locus/Gene-commentary/Gene-commentary_accession/text()")
    seq_from = root.xpath(".//Entrezgene_locus/Gene-commentary/Gene-commentary_seqs//Seq-interval_from/text()")
    seq_to = root.xpath(".//Entrezgene_locus/Gene-commentary/Gene-commentary_seqs//Seq-interval_to/text()")
    gene_length = abs(int(seq_from[0])-int(seq_to[0]))+1
    rows.append([seq_accession[0], int(seq_from[0]), int(seq_to[0]), gene_length])

In [ ]:
sacc_table = pd.DataFrame(rows, columns = ["accession", "from", "to", "len"])

sacc_table

In [ ]:
fig, axes = plt.subplots()
axes.hist(sacc_table.len, bins=20)




 ## 12. **Buscar entradas con anotaciones específicas**
 Buscar 10 entradas que tengan el qualifier
 `/product="cytochrome c oxidase subunit I"`
  y guarda sus FASTA en archivos separados.

In [ ]:
# /product == Prot Name o Gene Name

term = '"cytochrome c oxidase subunit I"[Prot name]'

search = eSearch(term, "nuccore", retmax=10)

identifiers = [x.value for x in jp.parse("$..idlist[*]").find(search)]

for i in identifiers:
    data = efetch(i, "nuccore", format="fasta")
    with open(f"{i}.fasta", "w", encoding="utf8") as fout:
        fout.write(data)
    data = efetch(i, "nuccore", format="genbank")
    with open(f"{i}.gb", "w", encoding="utf8") as fout:
        fout.write(data)



 ## 13. **Número de registros en el tiempo**
 Usa `esearch` para contar el número de registros en GenBank de secuencias de
 genomas completos de HIV1 por mes desde 1986 hasta la actualidad. Elimina
 todas aquellas que en el título diga "UNVERIFIED".

In [ ]:
import itertools

counts = []
for year, month in itertools.product(range(1986,2026), range(1,13)):
    term = f'txid11676[organism] AND "{year}/{month:02d}"[Publication Date] AND "complete genome"[Title]'
    time.sleep(0.01)
    search = eSearch(term, "nuccore")
    c_counts = int(jp.parse("$..count").find(search)[0].value)
    counts.append([f"{year}/{month:02d}", c_counts])

counts

In [ ]:
fig, axes = plt.subplots(figsize=(12, 8))
axes.plot(
    range(len(counts)),
    [x[1] for x in counts]
)
axes.set_xticks([x for x in range(len(counts)) if x % 12 == 0])
_ = axes.set_xticklabels(
    [x[0]  for x in counts if x[0].endswith("01")],
    rotation=90
)



 ## 14. **Comparar Tabla de CDS**
 Descarga las tablas de features para todos genomas mitocondriales completos
 de C. elegans y compara sus CDS. ¿Cuántos son iguales?
 ¿Cuántos son diferentes?

In [ ]:
term = '"Caenorhabditis elegans"[Organism] AND "complete genome"[title] AND mitochondrion[filter]'

search = eSearch(term, "nuccore")

identifiers = [x.value for x in jp.parse("$..idlist[*]").find(search)]
identifiers



In [ ]:
all_ft = []
for i in identifiers:
    data = efetch(i, "nuccore", "ft")
    all_ft.append(data)

In [ ]:

def ft_to_dataframe(ft:str) -> Optional[pd.DataFrame]:
    lines = ft.split("\n")
    lines = [l for l in lines if not l.startswith(">")]
    last_feat = None
    new_lines = []
    for line in lines:
        fields = line.split("\t")
        if len(fields) == 3:
            last_feat = fields
            continue
        if len(fields) == 5:
            fields = last_feat + fields[3:5]
            new_lines.append(fields)
    if not new_lines:
        return None
    return pd.DataFrame(
        new_lines,
        columns=["start", "end", "feature", "qualifier", "value"]
    )

ft_tables = [
    ft_to_dataframe(x)
    for x in all_ft
]

ft_tables



## 15. **Recuperar genes en region cromosómica**

Recupera los nombre de todos los genes que se encuentran en el cromosoma 10,
entre las posiciones 100000 y 1000000 en humanos y en ratón.



In [15]:
term = '("homo sapiens"[Organism] OR "mus musculus"[ORGANISM]) AND 10[Chromosome] AND 100000:1000000[Base Position]'

search = eSearch(term, "gene", retmax=100)

identifiers = [x.value for x in jp.parse("$..idlist[*]").find(search)]

In [ ]:
summaries = eSummary(",".join(identifiers), 'gene')

In [21]:
data = []
for i in identifiers:
    query = f'$..["{i}"].uid'
    uid = jp.parse(query).find(summaries)[0].value
    query = f'$..["{i}"].description'
    description = jp.parse(query).find(summaries)[0].value
    query = f'$..["{i}"].name'
    name = jp.parse(query).find(summaries)[0].value
    organism = jp.parse(f'$..["{i}"].organism.scientificname').find(summaries)[0].value
    data.append((uid, name, description, organism))

chr10_table = pd.DataFrame(data, columns=["uid", "name", "description", "scientific_name"])
chr10_table


,uid,name,description,scientific_name
0,23560,GTPBP4,GTP binding protein 4,Homo sapiens
1,10771,ZMYND11,zinc finger MYND-type containing 11,Homo sapiens
2,23185,LARP4B,La ribonucleoprotein 4B,Homo sapiens
3,22982,DIP2C,disco interacting protein 2 homolog C,Homo sapiens
4,100847086,MIR5699,microRNA 5699,Homo sapiens
...,...,...,...,...
76,107984285,LOC107984285,uncharacterized LOC107984285,Homo sapiens
77,107984191,LOC107984191,uncharacterized LOC107984191,Homo sapiens
78,107984190,LOC107984190,uncharacterized LOC107984190,Homo sapiens
79,105376341,LOC105376341,uncharacterized LOC105376341,Homo sapiens


## 16. **Recuperar variantes**

Recuperar cuantas variantes existen para el gen FUS humano en la base de datos
dbVar, que tengan una frecuencia alélica global menor al 1%.



In [25]:

term = 'FUS[Gene] AND human[ORGANSIM] AND 0:0.01[Global Allele Frequency]'

search = eSearch(term, "dbVar")

total_variants = jp.parse("$..count").find(search)[0].value
total_variants

'58'